# ------------------------------------------------------------------------------
Names:
Zilal Eiz Al Din && Payel Chakraborty

NetIDs:
zelalae2  &&  payelc2

Purpose:
This file contains Paper Name, their Pretrained Models, Environment and Prerequisites and Data processing.

Dataset:
HurtfulWordsDataset (MIMIC-III)

Paper Reference:
Hurtful Words: Quantifying Biases in Clinical Contextual Word Embeddings

The paper: https://arxiv.org/abs/2003.11515
Github Repo of the Original paper: https://github.com/MLforHealth/HurtfulWords
Github Repo of the Reproduction: https://github.com/zelal-Eizaldeen/DLH-Project-Reproduce-HurtfulWords

However, these steps have been obtainwd from https://github.com/YerevaNN/mimic3-benchmarks  to build the benchmark. It assumes that you already have MIMIC-III dataset (lots of CSV files) on the disk.

Usage: From the Github link mentioend above, "After all the below commands are done, there will be a directory data/{task} for each created benchmark task. These directories have two subdirectories: train and test. Each of them contains a bunch of ICU stays and one file with name listfile.csv, which lists all samples in that particular set. Each row of listfile.csv has the following form: icu_stay, period_length, label(s). A row specifies a sample for which the input is the collection of ICU event of icu_stay that occurred in the first period_length hours of the stay and the target is/are label(s). In in-hospital mortality prediction task period_length is always 48 hours, so it is not listed in corresponding listfiles."

Please note that extract_subjects.py has been changed as per our requirement, so it has been uploaded to our Github of reproduction.
# ------------------------------------------------------------------------------

# These steps have been run to create the "mimic-benchmark" directory and its content as per the directions on https://github.com/YerevaNN/mimic3-benchmarks.

# Mounting the Google Colab

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
# !git clone https://github.com/YerevaNN/mimic3-benchmarks/

Cloning into 'mimic3-benchmarks'...
remote: Enumerating objects: 1898, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 1898 (delta 58), reused 34 (delta 34), pack-reused 1820 (from 3)
Receiving objects: 100% (1898/1898), 16.94 MiB | 15.21 MiB/s, done.
Resolving deltas: 100% (1308/1308), done.


In [ ]:
%cd '/content/drive/MyDrive/Payel-DLH-related/mimic3-benchmarks/'

/content/drive/MyDrive/Payel-DLH-related/mimic3-benchmarks


In [ ]:
# import pandas as pd

# # Load your file (replace with your actual file path)
# df = pd.read_csv('/content/drive/MyDrive/Payel-DLH-related/DataFiles/cleaned_data/data.csv')

# # Replace 'subject_id' with the actual column name that identifies subjects
# num_unique_subjects = df['subject_id'].nunique()
# print("Number of unique subjects:", num_unique_subjects)

Number of unique subjects: 1411


#Benchmark Execution Steps

In [ ]:
!pip install -r requirements.txt

In [ ]:
%cd data
!ls

/content/drive/MyDrive/Payel-DLH-related/mimic3-benchmarks/data
decompensation	in-hospital-mortality  root


# 1-The following command takes MIMIC-III CSVs, generates one directory per SUBJECT_ID and writes ICU stay information to data/{SUBJECT_ID}/stays.csv, diagnoses to data/{SUBJECT_ID}/diagnoses.csv, and events to data/{SUBJECT_ID}/events.csv. This step might take around an hour.
# Please note that the extract_subjects.py script has been changed in this context to retain only subjects that are within our "data.csv" that contains patients information with notes filtered to "Nursing", "Nursing/other", "Physician"", and Discharge Summary" and restricted to the first 48 hours of ICU stay.

In [ ]:
#The original unsampled data files
PATH_TO_CSVs = '/content/drive/MyDrive/Payel-DLH-related/DataFiles/mimic-iii-clinical-database-1.4'

#Zilal PAth


In [ ]:
!ls $PATH_TO_CSVs

ADMISSIONS.csv		   D_ICD_PROCEDURES.csv    OUTPUTEVENTS.csv
CALLOUT.csv		   D_ITEMS.csv		   PATIENTS.csv
CAREGIVERS.csv		   D_LABITEMS.csv	   PRESCRIPTIONS.csv
CHARTEVENTS.csv		   DRGCODES.csv		   PROCEDUREEVENTS_MV.csv
checksum_md5_unzipped.txt  ICUSTAYS.csv		   PROCEDURES_ICD.csv
checksum_md5_zipped.txt    INPUTEVENTS_CV.csv	   README.md
CPTEVENTS.csv		   INPUTEVENTS_MV.csv	   SERVICES.csv
DATETIMEEVENTS.csv	   LABEVENTS.csv	   SHA256SUMS.txt
D_CPT.csv		   LICENSE.txt		   TRANSFERS.csv
DIAGNOSES_ICD.csv	   MICROBIOLOGYEVENTS.csv
D_ICD_DIAGNOSES.csv	   NOTEEVENTS.csv


In [ ]:
!pwd

/content/drive/MyDrive/Payel-DLH-related/mimic3-benchmarks


In [ ]:
!python -m mimic3benchmark.scripts.extract_subjects $PATH_TO_CSVs data/root/

START:
	ICUSTAY_IDs: 61532
	HADM_IDs: 57786
	SUBJECT_IDs: 46476
REMOVE ICU TRANSFERS:
	ICUSTAY_IDs: 55830
	HADM_IDs: 52834
	SUBJECT_IDs: 43277
REMOVE MULTIPLE STAYS PER ADMIT:
	ICUSTAY_IDs: 50186
	HADM_IDs: 50186
	SUBJECT_IDs: 41587
REMOVE PATIENTS AGE < 18:
	ICUSTAY_IDs: 42276
	HADM_IDs: 42276
	SUBJECT_IDs: 33798
Using only 1524 ICU stays and 1411 unique patients.
Breaking up stays by subjects: 100% 1411/1411 [00:14<00:00, 99.43it/s] 
Breaking up diagnoses by subjects: 100% 1411/1411 [00:12<00:00, 115.34it/s]
Processing CHARTEVENTS table: 100% 330712483/330712484 [30:09<00:00, 182715.33it/s]
Processing LABEVENTS table: 100% 27854055/27854056 [02:28<00:00, 187239.63it/s]
Processing OUTPUTEVENTS table: 100% 4349218/4349219 [01:04<00:00, 67372.68it/s] 


# 2-The following command attempts to fix some issues (ICU stay ID is missing) and removes the events that have missing information. About 80% of events remain after removing all suspicious rows

In [ ]:
# !python -m mimic3benchmark.scripts.validate_events ./data/root/
!python -m mimic3benchmark.scripts.validate_events data/root/

Namespace(subjects_root_path='data/root/')
Iterating over subjects: 100% 1411/1411 [01:04<00:00, 21.94it/s]
n_events: 6637992
empty_hadm: 161247
no_hadm_in_stay: 2123999
no_icustay: 413056
recovered: 413056
could_not_recover: 0
icustay_missing_in_stays: 310324


In [ ]:
# !ls data/root/23987/events.csv

diagnoses.csv  stays.csv


In [ ]:
!ls -l data/root | grep ^d | wc -l

1411


# 3- The next command breaks up per-subject data into separate episodes (pertaining to ICU stays). Time series of events are stored in {SUBJECT_ID}/episode{#}_timeseries.csv (where # counts distinct episodes) while episode-level information (patient age, gender, ethnicity, height, weight) and outcomes (mortality, length of stay, diagnoses) are stores in {SUBJECT_ID}/episode{#}.csv. This script requires two files, one that maps event ITEMIDs to clinical variables and another that defines valid ranges for clinical variables (for detecting outliers, etc.).

In [ ]:
!python -m mimic3benchmark.scripts.extract_episodes_from_subjects data/root/

Streaming output truncated to the last 5000 lines.
  labels[l] = 0
/content/drive/MyDrive/Payel-DLH-related/mimic3-benchmarks/mimic3benchmark/preprocessing.py:80: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  labels[l] = 0
/content/drive/MyDrive/Payel-DLH-related/mimic3-benchmarks/mimic3benchmark/preprocessing.py:80: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  labels[l] = 0
/content/drive/MyDrive/Payel-DLH-related/mimic3-benchmarks/mimic3benchmark/preprocessing.py:80: PerformanceWarning: DataFrame is highly fragmented.  This i

#4- The next command splits the whole dataset into training and testing sets. Note that the train/test split is the same of all tasks.

In [ ]:
!python -m mimic3benchmark.scripts.split_train_and_test data/root/

#5- The following commands will generate task-specific datasets, which can later be used in models. These commands are independent, if you are going to work only on one benchmark task, you can run only the corresponding command.

In [ ]:
# !python -m mimic3benchmark.scripts.create_in_hospital_mortality data/root/ data/in-hospital-mortality/
# !python -m mimic3benchmark.scripts.create_decompensation data/root/ data/decompensation/
!python -m mimic3benchmark.scripts.create_length_of_stay data/root/ data/length-of-stay/
!python -m mimic3benchmark.scripts.create_phenotyping data/root/ data/phenotyping/
!python -m mimic3benchmark.scripts.create_multitask data/root/ data/multitask/

Iterating over patients in test:  61% 102/166 [01:32<00:51,  1.24it/s]
	(empty label file) 11245 episode2_timeseries.csv
Iterating over patients in test: 100% 166/166 [02:26<00:00,  1.13it/s]
Number of created samples: 10715
Iterating over patients in train:   6% 69/1227 [01:15<26:56,  1.40s/it]
	(no events in ICU)  2601 episode1_timeseries.csv
Iterating over patients in train:  33% 407/1227 [05:06<05:49,  2.35it/s]
	(empty label file) 12607 episode2_timeseries.csv
Iterating over patients in train:  41% 499/1227 [06:05<08:06,  1.50it/s]
	(no events in ICU)  16680 episode3_timeseries.csv
Iterating over patients in train:  43% 525/1227 [06:24<05:59,  1.95it/s]
	(empty label file) 17269 episode2_timeseries.csv
Iterating over patients in train:  50% 613/1227 [07:27<07:25,  1.38it/s]
	(no events in ICU)  18552 episode1_timeseries.csv
Iterating over patients in train:  63% 769/1227 [09:05<03:39,  2.09it/s]
	(empty label file) 23407 episode2_timeseries.csv
Iterating over patients in train: 10

In [ ]:
# %cd '/content/drive/MyDrive/Payel-DLH-related/mimic3-benchmarks/data/phenotyping/train'
df = pd.read_csv('/content/drive/MyDrive/Payel-DLH-related/mimic3-benchmarks/data_bkp/phenotyping/train/listfile.csv')
df['subject_id'] = df['stay'].str.split('_').str[0]
df['subject_id'].nunique()

2018

After the above commands are done, there will be a directory data/{task} for each created benchmark task. These directories have two subdirectories: train and test. Each of them contains a bunch of ICU stays and one file with name listfile.csv, which lists all samples in that particular set. Each row of listfile.csv has the following form: icu_stay, period_length, label(s). A row specifies a sample for which the input is the collection of ICU event of icu_stay that occurred in the first period_length hours of the stay and the target is/are label(s). In in-hospital mortality prediction task period_length is always 48 hours, so it is not listed in corresponding listfiles.